In [3]:
import gensim
import pandas as pd
from gensim import corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric
from gensim.models import LdaModel
from gensim.parsing.preprocessing import preprocess_string
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import os

In [7]:
directory = "/Users/diegojimenez/Documents/GitHub/computational_ds/data/output"

texts=[]
# Iterar sobre los archivos CSV en la carpeta
for archivo_csv in os.listdir(directory):
    if archivo_csv.endswith('.csv'):
        # Leer el archivo CSV
        ruta_completa = os.path.join(directory, archivo_csv)
        df = pd.read_csv(ruta_completa)

        # Reemplaza 'nombre_de_la_columna' con el nombre real de la columna que contiene los textos
        columna_textos = 'Text'

        # Agregar los textos de la columna a la lista
        texts.extend(df[columna_textos].tolist())

# Imprimir la lista de textos
print(texts)








In [10]:
df=pd.read_csv("")

In [12]:
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    # Tokenize the text
    tokens = simple_preprocess(text)
    # Remove stopwords and other unwanted tokens
    custom_filters = [strip_tags, strip_punctuation, strip_numeric]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = preprocess_string(" ".join(tokens), custom_filters)
    return tokens

In [14]:
# Create a dictionary from the documents
news = df_2014["text_clean"].tolist()
dictionary = corpora.Dictionary(news)
# Create a corpus from the documents
corpus = [dictionary.doc2bow(doc) for doc in news]

In [15]:
lda_model = LdaModel(corpus, num_topics=10, id2word=dictionary, passes=10)

In [21]:
import pyLDAvis
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)